# Joining Data with Pandas

## 0 - Setup Environment

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

## 1 - Data Merging Basics

### Inner Joins | DF Loads: taxi_owners, taxi_veh, census, wards, biz_owners, licenses

##### Merge the taxi_owners and taxi_veh tables setting a suffix for columns w/ the same names

In [ ]:
taxi_owners = pd.read_pickle(r'datasets/taxi_owners.p')
taxi_veh = pd.read_pickle(r'datasets/taxi_vehicles.p')

taxi_own_veh = taxi_owners.merge(taxi_veh, on='vid', suffixes=('_own', '_veh'))
print(taxi_own_veh.columns)

##### Inner joins and number of rows returned

In [ ]:
census = pd.read_pickle(r'datasets/census.p')
wards = pd.read_pickle(r'datasets/ward.p')

wards_census = wards.merge(census, on='ward')
print(f'wards_census table shape: {wards_census.shape}')

##### One-to-many merge

In [ ]:
biz_owners = pd.read_pickle(r'datasets/business_owners.p')
licenses = pd.read_pickle(r'datasets/licenses.p')

licenses_owners = licenses.merge(biz_owners, on = 'account')

# Group the results by title then count the number of accounts
# NOTE: a dict {columm:f(x)}, i.e., {'account':'count'}, can be passed to .agg(...) to indicate different f(x)s for different columns
counted_df = licenses_owners.groupby('title').agg({'account':'count'})
sorted_df = counted_df.sort_values('account', ascending = False)
print(sorted_df.head())

### Merging Multiple DataFrames | DFs Loads: cal, ridership, stations, land_use

##### Merge 3 DataFrames and Create Complex Filter Criteria

In [ ]:
cal = pd.read_pickle(r'datasets/cta_calendar.p')
ridership = pd.read_pickle(r'datasets/cta_ridership.p')
stations = pd.read_pickle(r'datasets/stations.p')

ridership_cal_stations = ridership.merge(cal, on=['year','month','day']).merge(stations, on='station_id')
filter_criteria = ((ridership_cal_stations['month'] == 7) 
                   & (ridership_cal_stations['day_type'] == 'Weekday') 
                   & (ridership_cal_stations['station_name'] == 'Wilson'))
print(ridership_cal_stations.loc[filter_criteria, 'rides'].sum())

##### One-to-Many Merge with Multiple DataFrames - 'SQL-like' Results, i.e., columns passed to .groupby(...) are NOT converted to a MultIndex/Index

In [ ]:
land_use = pd.read_pickle(r'datasets/land_use.p')

land_cen_lic = land_use.merge(census, on='ward').merge(licenses, on='ward', suffixes=('_cen','_lic'))
# as_index = False is responsible for the 'SQL-like' results
pop_vac_lic = land_cen_lic.groupby(['ward', 'pop_2010', 'vacant'], as_index = False).agg({'account':'count'})
sorted_pop_vac_lic = pop_vac_lic.sort_values(['vacant', 'account', 'pop_2010'], ascending = [False, True, True])
print(sorted_pop_vac_lic.head())

## 2 - Merging Tables with Different Join Types

### Joins | DF Loads: financials, movies, taglines, movie_to_genres

##### Counting Missing Rows with Left Join

In [4]:
financials = pd.read_pickle(r'datasets/financials.p')
movies = pd.read_pickle(r'datasets/movies.p')

movies_financials = movies.merge(financials, on='id', how='left')

# using .sum(), count the number of NULLS in the budget column; using .count() is equivalent to select count(*)... w/o a where clause
number_of_missing_fin = movies_financials['budget'].isnull().sum()
print(number_of_missing_fin)

##### Enriching a dataset

In [ ]:
taglines = pd.read_pickle(r'datasets/taglines.p')

# notice that the .str accessor MUST be called multiple times; 1) .title.str.lower() 2) .title.str.lower().str.contains(...)
toy_story = movies[movies.title.str.lower().str.contains('toy story')]

print('========== Left Join Results ==========')
toystory_tag_lj = toy_story.merge(taglines, how = 'left', on = 'id')
print(toystory_tag_lj)

print('========== Inner Join Results ==========')
toystory_tag_ij = toy_story.merge(taglines, how = 'inner', on = 'id')
print(toystory_tag_ij)

##### Right join to Find Sci-Fi Movies

In [5]:
movie_to_genres = pd.read_pickle(r'datasets/movie_to_genres.p')
# attempted to re-create dataset not included w/ course material; new method saves DS to CSV, then prints it to screen to be copied
# scifi_movies = movie_to_genres.query('genre == "Science Fiction"').merge(movies, left_on = 'movie_id', right_on = 'id')[movie_to_genres.columns]
# action_movies = movie_to_genres.query('genre == "Action"').merge(movies, left_on = 'movie_id', right_on = 'id')[movie_to_genres.columns]
scifi_movies = pd.read_csv(r'datasets/scifi_movies.csv')
action_movies = pd.read_csv(r'datasets/action_movies.csv')
action_scifi = action_movies.merge(scifi_movies, on='movie_id', how='right',suffixes = ['_act', '_sci'])

scifi_only = action_scifi[action_scifi['genre_act'].isnull()]
movies_and_scifi_only = movies.merge(scifi_only, left_on = 'id', right_on ='movie_id')

print(movies_and_scifi_only.head())
print(movies_and_scifi_only.shape)

##### Popular Genres with Right Join

In [ ]:
# top 10 most popular movies
pop_movies = movies.sort_values('popularity', ascending = False).head(10)
genres_movies = movie_to_genres.merge(pop_movies, how='right', left_on = 'movie_id', right_on = 'id')
genre_count = genres_movies.groupby('genre').agg({'id':'count'})

genre_count.plot(kind='bar')
plt.show()

### Index merge for movie ratings | DF Loads: ratings

In [ ]:
ratings = pd.read_pickle(r'datasets/ratings.p')

ratings.set_index('id', inplace = True)
movies.set_index('id', inplace = True)
movies_ratings = movies.merge(ratings, on = 'id', how = 'left')

print(movies_ratings.head())

## 3 - Advance Merging and Concatenation

### Advanced Joins

##### Semi-Join, i.e., Inner Join 2 DFs, Keeping Only UNIQUE Rows from Left DF

In [ ]:
# attempted to re-create dataset not included w/ course material; new method saves DS to CSV, then prints it to screen to be copied
from sqlite3_conn_mgr import sqlite3_conn_mgr

with sqlite3_conn_mgr(r'datasets/chinook.db') as conn:
    genres = pd.read_sql_query('select * from genres', conn)
    # top_10_tracks = pd.read_sql_query('select t.* from tracks t inner join invoice_items i on i.TrackId = t.TrackId order by i.Quantity desc limit 10', conn)
    top_10_tracks = pd.read_sql_query('select * from tracks where TrackId <= 10', conn)

# print(f'{"="*10} genres.info() {"="*10}')
# genres.info()
# print(f'{"="*10} tracks.info() {"="*10}')
# top_10_tracks.info()

genres_tracks = genres.merge(top_10_tracks, on = 'GenreId')
print(genres_tracks.head())
# semi-join returns only matching rows from the left DF
top_genres = genres[genres.GenreId.isin(genres_tracks.GenreId)]
print(top_genres.head())

##### Anti-Join, i.e., Left Join 2 DFs, Keeping ...

In [ ]:
# attempted to re-create dataset not included w/ course material; new method saves DS to CSV, then prints it to screen to be copied
# with sqlite3_conn_mgr(r'datasets/chinook.db') as conn:
#     employees = pd.read_sql_query('select * from employees', conn)
#     # top_10_tracks = pd.read_sql_query('select t.* from tracks t inner join invoice_items i on i.TrackId = t.TrackId order by i.Quantity desc limit 10', conn)
#     top_cust = pd.read_sql_query('select * from customers where CustomerId < 60', conn)

# empl_cust = employees.merge(top_cust, how = 'left', left_on = 'EmployeeId', right_on = 'SupportRepId', indicator = True)
# srid_list = empl_cust.loc[empl_cust._merge == 'left_only', 'SupportRepId']
# print(employees[employees.EmployeeId.isin(srid_list)])

employees = pd.read_csv(r'datasets/employees.csv')
top_cust = pd.read_csv(r'datasets/top_cust.csv')
empl_cust = employees.merge(top_cust, on = 'srid', how = 'left', indicator = True)
# Select the srid column where _merge is left_only
srid_list = empl_cust.loc[empl_cust['_merge'] == 'left_only', 'srid']

# Get employees not working with top customers
print(employees[employees.srid.isin(srid_list)])

##### Semi-Join

In [ ]:
non_mus_tcks = pd.read_csv(r'datasets/non_mus_tcks.csv')
top_invoices = pd.read_csv(r'datasets/top_invoices.csv')
genres = pd.read_csv(r'datasets/genres.csv')

# Merge the non_mus_tck and top_invoices tables on tid
tracks_invoices = non_mus_tcks.merge(top_invoices, on = 'tid')

# Use .isin() to subset non_mus_tcks to rows with tid in tracks_invoices
top_tracks = non_mus_tcks[non_mus_tcks['tid'].isin(tracks_invoices.tid)]

# Group the top_tracks by gid and count the tid rows
cnt_by_gid = top_tracks.groupby(['gid'], as_index=False).agg({'tid':'count'})

# Merge the genres table to cnt_by_gid on gid and print
print(cnt_by_gid.merge(genres, on = 'gid'))

### Concatenating DataFrames

##### Vertical Concatenation

In [ ]:
tracks_master = pd.read_csv(r'datasets/tracks_master.csv')
tracks_ride = pd.read_csv(r'datasets/tracks_ride.csv')
tracks_st = pd.read_csv(r'datasets/tracks_st.csv')

tracks_from_albums = pd.concat([tracks_master, tracks_ride, tracks_st], sort=True)
print(tracks_from_albums)

##### Concatenate, Ignoring Existing Indexes

In [ ]:
# Concatenate the tracks so the index goes from 0 to n-1
tracks_from_albums = pd.concat([tracks_master, tracks_ride, tracks_st], ignore_index = True, sort = True)
print(tracks_from_albums)

##### Concatenate DFs with Different Columns - Keep Only Common Columns

In [ ]:
# Concatenate the tracks, show only columns names that are in all tables
tracks_from_albums = pd.concat([tracks_master, tracks_ride, tracks_st], join = 'inner', sort = True)
print(tracks_from_albums)

##### Concatenating with Keys

In [ ]:
inv_jul = pd.read_csv(r'datasets/inv_jul.csv')
inv_aug = pd.read_csv(r'datasets/inv_aug.csv')
inv_sep = pd.read_csv(r'datasets/inv_sep.csv')

# Concatenate the tables and add keys
inv_jul_thr_sep = pd.concat([inv_jul, inv_aug, inv_sep], keys=['7Jul', '8Aug', '9Sep'])

# Group the invoices by the index keys and find avg of the total column
avg_inv_by_month = inv_jul_thr_sep.groupby(level=0).agg({'total': 'mean'})

# Bar plot of avg_inv_by_month
avg_inv_by_month.plot(kind = 'bar')
plt.show()

##### Concatenate and merge to find common songs

In [ ]:
classic_18 = pd.read_csv(r'datasets/classic_18.csv')
classic_19 = pd.read_csv(r'datasets/classic_19.csv')
pop_18 = pd.read_csv(r'datasets/pop_18.csv')
pop_19 = pd.read_csv(r'datasets/pop_19.csv')

# Concatenate the classic tables vertically
classic_18_19 = pd.concat([classic_18, classic_19], ignore_index = True)

# Concatenate the pop tables vertically
pop_18_19 = pd.concat([pop_18, pop_19], ignore_index = True)

# Merge classic_18_19 with pop_18_19
classic_pop = classic_18_19.merge(pop_18_19, on = 'tid', how = 'inner')

# Using .isin(), filter classic_18_19 rows where tid is in classic_pop
popular_classic = classic_18_19[classic_18_19.tid.isin(classic_pop.tid)]

# Print popular chart
print(popular_classic)

## 4 - Querying and Manipulating DataFrames

##### Shaping Data with `.melt()`

In [ ]:
ur_wide = ur_wide = pd.read_csv(r'datasets/ur_wide.csv')
# following 2 lines are required for datacamp code to function
ur_wide.drop('origin_idx', axis = 1, inplace = True)

# unpivot everything besides the year column
ur_tall = ur_wide.melt(id_vars = ['year'], var_name = ['month'], value_name = 'unempl_rate')

# Create a date column using the month and year columns of ur_tall
ur_tall['date'] = pd.to_datetime(ur_tall.year.astype(str) + '-' + ur_tall.month)

# Sort ur_tall by date in ascending order
ur_sorted = ur_tall.sort_values(['date'])

# Plot the unempl_rate by date
ur_sorted.plot(x = 'date', y = 'unempl_rate')
plt.show()

##### Shaping Data with `.melt()` and Filtering with `.query()`

In [ ]:
ten_yr = pd.read_csv(r'datasets/ten_yr.csv')
dji = pd.read_csv(r'datasets/dji.csv')

# Use melt on ten_yr, unpivot everything besides the metric column
bond_perc = ten_yr.melt(id_vars = ['metric'], var_name = ['date'], value_name = 'close')

# Use query on bond_perc to select only the rows where metric=close
bond_perc_close = bond_perc.query('metric == "close"')

# Merge (ordered) dji and bond_perc_close on date with an inner join
dow_bond = pd.merge_ordered(dji, bond_perc_close, on = 'date', how = 'inner', suffixes = ('_dow', '_bond'))

# Plot only the close_dow and close_bond columns
dow_bond.plot(y = ['close_dow', 'close_bond'], x='date', rot=90)
plt.show()

In [2]:
from zip_util import compress_folder, decompress_folder
import os, sys
from pathlib import Path

cwd = Path(os.getcwd())
datasets = cwd.joinpath('datasets')
archive = cwd.joinpath('pd_datasets.zip')
compress_folder(datasets, archive, True)

Compressing /work/files/workspace/datasets to /work/files/workspace/pd_datasets.zip
Compressed /work/files/workspace/datasets to /work/files/workspace/pd_datasets.zip
